In [1]:
pip install pyngrok

Note: you may need to restart the kernel to use updated packages.


In [7]:
%%writefile app.py

import streamlit as st
import cv2 as cv
import numpy as np
import torch
import torchvision
from torchvision import transforms as T
from PIL import Image
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


label_dict = {
    0: "1.000",
    1: "2.000",
    2: "5.000",
    3: "10.000",
    4: "20.000",
    5: "50.000",
    6: "75.000",
    7: "100.000"
}


# Fungsi untuk memuat model
def load_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=8)
    model.load_state_dict(torch.load('rupiah_banknotes_model_88Accuracy.pth', map_location=torch.device('cpu')))
    model.eval()
    return model

# Fungsi untuk melakukan deteksi objek
def detect_objects(image, model):
    transform = T.Compose([T.ToTensor()])
    image_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        predictions = model(image_tensor)

    return predictions[0]

# Fungsi untuk menampilkan hasil deteksi
def draw_boxes(image, predictions, threshold=0.5):
    boxes = predictions['boxes'].numpy()
    scores = predictions['scores'].numpy()
    labels = predictions['labels'].numpy()

    for box, score, label in zip(boxes, scores, labels):
        if score >= threshold:
            x1, y1, x2, y2 = box
            descriptive_label = label_dict.get(label, str(label))
            cv.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv.putText(image, f'{descriptive_label}: {score:.2f}', (int(x1), int(y1) - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9,
                       (255, 0, 0), 2)

    return image

# Streamlit UI
st.title("Real-Time Rupiah Banknotes Detection with Faster R-CNN")
st.write("Upload an image to detect objects in real-time.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    image_np = np.array(image)

    st.image(image, caption='Uploaded Image', use_column_width=True)
    st.write("")
    st.write("Detecting objects...")

    # Muat model
    model = load_model()

    # Lakukan deteksi objek
    predictions = detect_objects(image, model)

    # Gambar kotak deteksi pada gambar asli
    result_image = draw_boxes(image_np.copy(), predictions)

    st.image(result_image, caption='Detected Objects', use_column_width=True)

st.write("")
st.write("Real-Time Webcam Object Detection")

if 'webcam_active' not in st.session_state:
    st.session_state.webcam_active = False

# Fungsi untuk menjalankan deteksi objek dengan webcam
def run_webcam(camera_id):
    cap = cv.VideoCapture(camera_id)
    model = load_model()
    stframe = st.empty()

    while st.session_state.webcam_active:
        ret, frame = cap.read()
        if not ret:
            break

        # Konversi frame ke format PIL
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame_rgb)

        # Lakukan deteksi objek
        predictions = detect_objects(frame_pil, model)

        # Gambar kotak deteksi pada frame asli
        frame_result = draw_boxes(frame, predictions)

        stframe.image(frame_result, channels="BGR", use_column_width=True)

    cap.release()
    cv.destroyAllWindows()

camera_id = st.selectbox("Select Camera", [0, 1, 2, 3, 4])  # Anda bisa menambahkan lebih banyak ID kamera jika diperlukan

if not st.session_state.webcam_active:
    if st.button('Start Webcam'):
        st.session_state.webcam_active = True
        run_webcam(camera_id)
else:
    if st.button('Stop Webcam'):
        st.session_state.webcam_active = False


Overwriting app.py


In [8]:
!streamlit run app.py & npx localtunnel --port 8501

^C
